# Segunda Tarefa do Titanic

In [16]:
import pandas as pd
#train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]
X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True):
        self.excluirName = excluirName
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        return self
    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)
        return Xdrop


In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()


In [19]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()


In [20]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier())
])

parametros = {
    'atributosDesejados__excluirName': [True, False],
    'classificador__max_depth': [5],
    #'classificador__criterion': ['gini', 'entropy'],
    #'classificador__max_features': ['sqrt', 'log2'],
    #'classificador__class_weight': [None, 'balanced', 'balanced_subsample']
}
modelo = GridSearchCV(pipetotal, param_grid=parametros, n_jobs=-1)

scores = cross_validate(modelo, X, y, cv=RepeatedKFold(n_splits=7, n_repeats=10, random_state=None))
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

(array([0.84375   , 0.8515625 , 0.82677165, 0.80314961, 0.79527559,
        0.81889764, 0.87401575, 0.8203125 , 0.8046875 , 0.84251969,
        0.8503937 , 0.8503937 , 0.84251969, 0.79527559, 0.84375   ,
        0.859375  , 0.86614173, 0.82677165, 0.78740157, 0.78740157,
        0.77165354, 0.8515625 , 0.8359375 , 0.82677165, 0.77952756,
        0.79527559, 0.81889764, 0.81889764, 0.8046875 , 0.7578125 ,
        0.83464567, 0.83464567, 0.82677165, 0.87401575, 0.84251969,
        0.8515625 , 0.796875  , 0.81102362, 0.80314961, 0.8503937 ,
        0.85826772, 0.82677165, 0.8203125 , 0.8359375 , 0.85826772,
        0.82677165, 0.84251969, 0.77952756, 0.81102362, 0.875     ,
        0.8359375 , 0.78740157, 0.81102362, 0.81889764, 0.80314961,
        0.77165354, 0.8125    , 0.84375   , 0.90551181, 0.77165354,
        0.78740157, 0.8976378 , 0.80314961, 0.8125    , 0.7734375 ,
        0.88188976, 0.79527559, 0.83464567, 0.84251969, 0.82677165]),
 0.8246871484814398,
 0.031244065039984868)

In [22]:
modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)